In [ ]:
# URL страницы с отзывами
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd

url = "https://travel.yandex.ru/hotels/moscow/gamma-izmailovo/?adults=1&checkinDate=2024-11-14&checkoutDate=2024-11-23&childrenAges=&searchPagePollingId=e2e6db29c5fd2766cd7a5ad5ff35b910-0-newsearch&seed=portal-hotels-search"

# Заголовки для запроса
headers = {"User-Agent": "Mozilla/5.0", "Accept-Language": "ru-RU,ru;q=0.9,en;q=0.8"}

# Выполняем запрос к странице
response = requests.get(url, headers=headers)

# Парсим страницу с помощью BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")
# print(soup)
# Преобразуем весь HTML-код страницы в строку
html_text = soup.prettify()
data_list = []
# print(html_text.split("textReviews")[1], len(html_text.split("textReviews")[1]), html_text.split("textReviews")[1].find('"text":'))
pattern = re.compile(r'\{.*?"text":.*?\}')
matches = pattern.findall(html_text.split("textReviews")[1])
for match in matches:
    data = match.split('{"status":"ACCEPTED"},')

    if len(data) < 2:
        continue
    # print(list(filter(lambda el: el.startswith('"snippet":') or  el.startswith('"text":'),data[1].split(','))))
    # print(data[1].split('","'))
    data = list(
        filter(
            lambda el: el.startswith('"snippet":') or el.startswith('text":'),
            data[1].split('","'),
        )
    )
    # print(data)
    try:
        snippet = data[0].split(":")[1].strip('"')
        text = data[1].split(":")[1].strip('"')
        print(snippet, text)
        if snippet is not None:
            # Добавляем данные в список
            data_list.append(
                {
                    # "snippet": snippet,
                    "text": snippet + text,
                }
            )
    except IndexError:
        continue

# Создаем DataFrame из списка
df = pd.DataFrame(data_list)

# Выводим DataFrame
df
